# Writing scripts for running the ChromBPNet pipeline
This notebook takes in a set of file paths and generates the necessary bash scripts to run through the ChromBPNet pipeline specified in the README file.

This notebook makes several assumptions

1. You have fragments files (BED files indicating the mapping of each read pair) for each group you want to train a model. This can be cell types, conditions, etc. or any combinations of these.
2. You have narrowPeak files for each group you want to train a model. Most of the time this should match grouping of the fragments files.
3. 

# Set-up

In [1]:
# Imports
import os
import glob
import yaml
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
# Input data paths
path_fragments = "/cellar/users/aklie/data/datasets/HPAP/fragments"
path_narrowPeaks = "/cellar/users/aklie/data/datasets/HPAP/peaks"

# Genome information
path_genome_fasta = "/cellar/users/aklie/data/ref/genomes/hg38/hg38.fa"
path_chromsizes = "/cellar/users/aklie/data/ref/genomes/hg38/hg38.chrom.sizes"
path_blacklist = "/cellar/users/aklie/data/ref/genomes/hg38/blacklist/blacklist.bed.gz"
path_fold_dir = "/cellar/users/aklie/data/ref/genomes/hg38/chrombpnet/splits"

# TF motif database
path_meme_file = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/ref/motifs.meme"

# Auxiliary scripts
path_wigToBigWig = "/cellar/users/aklie/opt/wigToBigWig"
path_contribution_averaging_script = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/scripts/average_importance_h5_over_folds.py"
path_pfm_script = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/scripts/modisco_to_pfm.py"
path_meme_script = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/scripts/pfm_to_meme.py"
path_variant_scorer = "/cellar/users/aklie/opt/variant-scorer/src/variant_scoring.py"

# Output path
path_out = "/cellar/users/aklie/data/datasets/HPAP/models"
os.makedirs(path_out, exist_ok=True)

In [3]:
# Params
name = "HPAP"

# Find files

In [4]:
# Get a dictionary of fragments files for each group of interest
fragments_dict = {}
for f in glob.glob(f"{path_fragments}/*.bed"):
    celltype = os.path.basename(f).split(".")[0]
    fragments_dict[celltype] = f
fragments_dict

{'Schwann': '/cellar/users/aklie/data/datasets/HPAP/fragments/Schwann.bed',
 'Ductal': '/cellar/users/aklie/data/datasets/HPAP/fragments/Ductal.bed',
 'Acinar': '/cellar/users/aklie/data/datasets/HPAP/fragments/Acinar.bed',
 'Delta': '/cellar/users/aklie/data/datasets/HPAP/fragments/Delta.bed',
 'Endothelial': '/cellar/users/aklie/data/datasets/HPAP/fragments/Endothelial.bed',
 'Q_Stellate': '/cellar/users/aklie/data/datasets/HPAP/fragments/Q_Stellate.bed',
 'A_Stellate': '/cellar/users/aklie/data/datasets/HPAP/fragments/A_Stellate.bed',
 'Gamma': '/cellar/users/aklie/data/datasets/HPAP/fragments/Gamma.bed',
 'Alpha': '/cellar/users/aklie/data/datasets/HPAP/fragments/Alpha.bed',
 'Beta': '/cellar/users/aklie/data/datasets/HPAP/fragments/Beta.bed',
 'Immune': '/cellar/users/aklie/data/datasets/HPAP/fragments/Immune.bed',
 'MUC5B_Ductal': '/cellar/users/aklie/data/datasets/HPAP/fragments/MUC5B_Ductal.bed'}

In [5]:
# Get a dictionary of narrowPeak files for each group
narrowPeaks_dict = {}
for f in glob.glob(f"{path_narrowPeaks}/*.narrowPeak"):
    celltype = os.path.basename(f).split("_peaks.narrowPeak")[0]
    narrowPeaks_dict[celltype] = f
narrowPeaks_dict

{'Delta': '/cellar/users/aklie/data/datasets/HPAP/peaks/Delta_peaks.narrowPeak',
 'Acinar': '/cellar/users/aklie/data/datasets/HPAP/peaks/Acinar_peaks.narrowPeak',
 'Alpha': '/cellar/users/aklie/data/datasets/HPAP/peaks/Alpha_peaks.narrowPeak',
 'Schwann': '/cellar/users/aklie/data/datasets/HPAP/peaks/Schwann_peaks.narrowPeak',
 'MUC5B_Ductal': '/cellar/users/aklie/data/datasets/HPAP/peaks/MUC5B_Ductal_peaks.narrowPeak',
 'Q_Stellate': '/cellar/users/aklie/data/datasets/HPAP/peaks/Q_Stellate_peaks.narrowPeak',
 'Beta': '/cellar/users/aklie/data/datasets/HPAP/peaks/Beta_peaks.narrowPeak',
 'Immune': '/cellar/users/aklie/data/datasets/HPAP/peaks/Immune_peaks.narrowPeak',
 'Gamma': '/cellar/users/aklie/data/datasets/HPAP/peaks/Gamma_peaks.narrowPeak',
 'Endothelial': '/cellar/users/aklie/data/datasets/HPAP/peaks/Endothelial_peaks.narrowPeak',
 'A_Stellate': '/cellar/users/aklie/data/datasets/HPAP/peaks/A_Stellate_peaks.narrowPeak',
 'Ductal': '/cellar/users/aklie/data/datasets/HPAP/peaks/

In [6]:
# Make sure keys are the same
assert set(fragments_dict.keys()) == set(narrowPeaks_dict.keys())

In [7]:
# Define the groups to train models on
groups = list(fragments_dict.keys())
groups

['Schwann',
 'Ductal',
 'Acinar',
 'Delta',
 'Endothelial',
 'Q_Stellate',
 'A_Stellate',
 'Gamma',
 'Alpha',
 'Beta',
 'Immune',
 'MUC5B_Ductal']

# Negatives

In [8]:
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/negatives.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/negatives"
name = "HPAP"

In [9]:
folds = 1
celltypes = []
fragments = []
peaks = []
output_dirs = []
fold_lst = []
for group in groups:
    for i in range(folds):
        outdir = f"{path_out}/{group}/fold_{i}/negatives"
        os.makedirs(outdir, exist_ok=True)
        celltypes.append(group)
        fragments.append(fragments_dict[group])
        peaks.append(narrowPeaks_dict[group])
        output_dirs.append(outdir)
        fold_lst.append(i)
print(len(celltypes), len(fragments), len(peaks), len(output_dirs), len(fold_lst))

12 12 12 12 12


In [10]:
# Print out celltypes in same way I would write a bash array so I can copy paste into a script
celltypes_str="celltypes=(\n"
for celltype in celltypes:
    celltypes_str += f"\t{celltype}\n"
celltypes_str += ")"
print(celltypes_str)

# Samme for peaks
peaks_str="peaks=(\n"
for peak in peaks:
    peaks_str += f"\t{peak}\n"
peaks_str += ")"
print(peaks_str)

# Samme for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

#  Samme for fold_lst
fold_lst_str="folds=(\n"
for fold in fold_lst:
    fold_lst_str += f"\t{fold}\n"
fold_lst_str += ")"
print(fold_lst_str)

celltypes=(
	Schwann
	Ductal
	Acinar
	Delta
	Endothelial
	Q_Stellate
	A_Stellate
	Gamma
	Alpha
	Beta
	Immune
	MUC5B_Ductal
)
peaks=(
	/cellar/users/aklie/data/datasets/HPAP/peaks/Schwann_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Ductal_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Acinar_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Delta_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Endothelial_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Q_Stellate_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/A_Stellate_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Gamma_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Alpha_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Beta_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Immune_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/MUC5B_Ductal_peaks.narrowPeak
)
output_dir

In [11]:
# Read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to run chrombpnet negatives command across multiple peaksets as a SLURM array job\n# USAGE: sbatch \\\n#--job-name=negatives \\\n#--partition carter-compute \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=16G \\\n#-n 1 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#negatives.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\n\n# file lists\n{}\n{}\n{}\n{}\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\nfold=${{folds[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# echo the celltype and peak\necho -e "Celltype: $celltype"\necho -e "Peakset: $peak"\necho -e "Fold: $fold"\necho -e "Output directory: $output_dir\\n"\n\n# make the output directory\nmkdir -p $output_dir\n\n# Run cmd\ncmd="chrombpnet

In [12]:
# Write the script
with open(f"{path_scripts}/{name}_negatives.sh", "w") as f:
    f.write(template.format(
        celltypes_str,
        peaks_str,
        fold_lst_str,
        output_dirs_str,
        path_genome_fasta,
        path_chromsizes,
        path_fold_dir,
        path_blacklist
    ))

# Bias models

In [13]:
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/bias_pipeline.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/bias_pipeline"
os.makedirs(path_scripts, exist_ok=True)
name = "HPAP"
beta = 0.3

In [14]:
# Find negatives
negatives = []
for outdir in output_dirs:
    group = outdir.split("/")[-3]
    negatives.append(f"{outdir}/{group}_negatives.bed")

# Printable list
negatives_str="negatives=(\n"
for negative in negatives:
    negatives_str += f"\t{negative}\n"
negatives_str += ")"
print(negatives_str)

negatives=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/negatives/Schwann_negatives.bed
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/negatives/Ductal_negatives.bed
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/negatives/Acinar_negatives.bed
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/negatives/Delta_negatives.bed
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/negatives/Endothelial_negatives.bed
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/negatives/Q_Stellate_negatives.bed
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/negatives/A_Stellate_negatives.bed
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/negatives/Gamma_negatives.bed
	/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/negatives/Alpha_negatives.bed
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/negatives/Beta_negatives.bed
	/cellar/users/aklie/data/datasets/HPAP/models/Imm

In [15]:
# Update output_dirs to replace "negatives" with bias_model/$beta
output_dirs = [f"{outdir.replace('/negatives', '')}/bias_model/$beta" for outdir in output_dirs]

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/Immune/fold_0/bias_model/$beta
	/cellar/users/aklie/data/datasets/HPAP/models/MUC5B_Ductal/fold_0/bias_model/$beta
)


In [16]:
# Same for fragments
fragments_str="fragments=(\n"
for fragment in fragments:
    fragments_str += f"\t{fragment}\n"
fragments_str += ")"
print(fragments_str)

fragments=(
	/cellar/users/aklie/data/datasets/HPAP/fragments/Schwann.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/Ductal.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/Acinar.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/Delta.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/Endothelial.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/Q_Stellate.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/A_Stellate.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/Gamma.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/Alpha.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/Beta.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/Immune.bed
	/cellar/users/aklie/data/datasets/HPAP/fragments/MUC5B_Ductal.bed
)


In [17]:
# Read in template as a string
template = template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to run chrombpnet bias model pipeline across multiple fragment files as a SLURM array job\n# USAGE: sbatch \\\n#--job-name=bias_pipeline \\\n#--account carter-gpu \\\n#--partition carter-gpu \\\n#--gpus=a30:1 \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=128G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#bias_pipeline.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import tensorflow as tf; print(tf.config.list_physical_devices(\'GPU\'))"\n\n# file lists\nbeta={}\n{}\n{}\n{}\n{}\n{}\n{}\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\nfragment=${{fragments[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\nfold=${{folds[$SLURM_ARRAY_TASK_ID - 1]}}\nnegative=${{negatives[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLU

In [18]:
# Write the script
with open(f"{path_scripts}/{name}_bias_pipeline.sh", "w") as f:
    f.write(template.format(
        beta,
        celltypes_str,
        fragments_str,
        peaks_str,
        fold_lst_str,
        negatives_str,
        output_dirs_str,
        path_genome_fasta,
        path_chromsizes,
        path_fold_dir,
    ))


# ChromBPNet models

In [19]:
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/chrombpnet_pipeline.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/chrombpnet_pipeline"
os.makedirs(path_scripts, exist_ok=True)
name = "HPAP"
beta = 0.3
folds = 1

In [20]:
# Find the bias models and put into matching list
bias_dir = "/cellar/users/aklie/data/datasets/HPAP/models/Beta"
bias_models = {}
f = glob.glob(f"{bias_dir}/fold_*/bias_model/{beta}/models/*bias.h5")
for model in f:
    fold = model.split("/")[-5]
    bias_models[fold] = model
bias_models_lst = [bias_models[f"fold_{fold}"] for fold in fold_lst]

# Same for bias_models
bias_models_str="bias_models=(\n"
for model in bias_models_lst:
    bias_models_str += f"\t{model}\n"
bias_models_str += ")"
print(bias_models_str)

bias_models=(
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/0.3/models/Beta_bias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/0.3/models/Beta_bias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/0.3/models/Beta_bias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/0.3/models/Beta_bias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/0.3/models/Beta_bias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/0.3/models/Beta_bias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/0.3/models/Beta_bias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/0.3/models/Beta_bias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/0.3/models/Beta_bias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/bias_model/0.3/models/Beta_bias.h5
	/cellar/users/aklie/data/datasets/HPAP/models

In [21]:
# Update output_dirs to be 
output_dirs = []
for group in groups:
    for i in range(folds):
        output_dirs.append(f"{path_out}/{group}/fold_{i}/chrombpnet/{beta}")

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Immune/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/MUC5B_Ductal/fold_0/chrombpnet/0.3
)


In [22]:
# Read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to run chrombpnet model pipeline across multiple fragment files as a SLURM array job\n# USAGE: sbatch \\\n#--job-name=chrombpnet_pipeline \\\n#--account carter-gpu \\\n#--partition carter-gpu \\\n#--gpus=a30:1 \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=128G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#chrombpnet_pipeline.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import tensorflow as tf; print(tf.config.list_physical_devices(\'GPU\'))"\n\n# file lists\nbeta={}\n{}\n{}\n{}\n{}\n{}\n{}\n{}\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\nfragment=${{fragments[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\nfold=${{folds[$SLURM_ARRAY_TASK_ID - 1]}}\nnegative=${{negatives[$SLURM_ARRAY_TASK_ID - 1]}}\nbias_model=${{bias_

In [23]:
# Write the script
with open(f"{path_scripts}/{name}_chrombpnet_pipeline.sh", "w") as f:
    f.write(template.format(
        beta,
        celltypes_str,
        fragments_str,
        peaks_str,
        fold_lst_str,
        negatives_str,
        bias_models_str,
        output_dirs_str,
        path_genome_fasta,
        path_chromsizes,
        path_fold_dir,
    ))

# Predictions

In [24]:
# Paths
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/predictions.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/predictions"
os.makedirs(path_scripts, exist_ok=True)
name = "HPAP"

In [25]:
# Find bias models
bias_models = []
for outdir in output_dirs:
    f = glob.glob(f"{outdir}/models/bias_model_scaled.h5")[0]
    #f = f"{outdir}/models/bias_model_scaled.h5"
    group = outdir.split("/")[-3]
    bias_models.append(f)

# Same for bias_models
bias_models_str="bias_models=(\n"
for model in bias_models:
    bias_models_str += f"\t{model}\n"
bias_models_str += ")"
print(bias_models_str)

bias_models=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/chrombpnet/0.3/models/bias_model_scaled.h5
	/cellar/users/aklie/data/datasets/HPAP

In [26]:
# Find full chrombpnet models
chrombpnet_models = []
for outdir in output_dirs:
    f = glob.glob(f"{outdir}//models/chrombpnet.h5")[0]
    #f = f"{outdir}/models/chrombpnet.h5"
    group = outdir.split("/")[-3]
    chrombpnet_models.append(f)

# Same for chrombpnet_models
chrombpnet_models_str="chrombpnet_models=(\n"
for model in chrombpnet_models:
    chrombpnet_models_str += f"\t{model}\n"
chrombpnet_models_str += ")"
print(chrombpnet_models_str)

chrombpnet_models=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3//models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3//models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3//models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3//models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3//models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3//models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3//models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3//models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/chrombpnet/0.3//models/chrombpnet.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/chrombpnet/0.3//models/chrom

In [27]:
# Find chrombpnet_nobias models
chrombpnet_nobias_models = []
for outdir in output_dirs:
    f = glob.glob(f"{outdir}/models/chrombpnet_nobias.h5")[0]
    #f = f"{outdir}/models/chrombpnet_nobias.h5"
    group = outdir.split("/")[-3]
    chrombpnet_nobias_models.append(f)

# Same for chrombpnet_nobias_models
chrombpnet_nobias_models_str="chrombpnet_nobias_models=(\n"
for model in chrombpnet_nobias_models:
    chrombpnet_nobias_models_str += f"\t{model}\n"
chrombpnet_nobias_models_str += ")"
print(chrombpnet_nobias_models_str)

chrombpnet_nobias_models=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/

In [28]:
# Add predictions to output_dirs
output_dirs = [f"{outdir}/predictions" for outdir in output_dirs]

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/predictions
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/predictions
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3/predictions
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3/predictions
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3/predictions
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3/predictions
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3/predictions
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3/predictions
	/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/chrombpnet/0.3/predictions
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/chrombpnet/0.3/predictions
	/cellar/users/aklie/data/datasets/HPAP/models/Immune/fold_0/chrombpnet/0.3/predictions
	/cellar/

In [29]:
# Read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to make predictions with a chrombpnet model\n# USAGE: sbatch \\\n#--job-name=predictions \\\n#--account carter-gpu \\\n#--partition carter-gpu \\\n#--gpus=a30:1 \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=32G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#predictions.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import tensorflow as tf; print(tf.config.list_physical_devices(\'GPU\'))"\n\n# file lists\n{}\n{}\n{}\n{}\n{}\n{}\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\nbias_model=${{bias_models[$SLURM_ARRAY_TASK_ID - 1]}}\nchrombpnet_model=${{chrombpnet_models[$SLURM_ARRAY_TASK_ID - 1]}}\nchrombpnet_nobias_model=${{chrombpnet_nobias_models[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM

In [30]:
# Write the script
with open(f"{path_scripts}/{name}_predictions.sh", "w") as f:
    f.write(template.format(
        celltypes_str,
        peaks_str,
        bias_models_str,
        chrombpnet_models_str,
        chrombpnet_nobias_models_str,
        output_dirs_str,
        path_genome_fasta,
        path_chromsizes,
    ))

# Averaging predictions

In [31]:
# Paths
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/average_predictions.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/predictions"
os.makedirs(path_scripts, exist_ok=True)
name = "HPAP"

In [32]:
# group outdirs by celltype
groups = {}
for outdir in output_dirs:
    group = outdir.split("/")[-5]
    if group not in groups:
        groups[group] = []
    groups[group].append(outdir)
groups

{'Schwann': ['/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/predictions'],
 'Ductal': ['/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/predictions'],
 'Acinar': ['/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3/predictions'],
 'Delta': ['/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3/predictions'],
 'Endothelial': ['/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3/predictions'],
 'Q_Stellate': ['/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3/predictions'],
 'A_Stellate': ['/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3/predictions'],
 'Gamma': ['/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3/predictions'],
 'Alpha': ['/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/chrombpnet/0.3/predictions'],
 'Beta': ['/cellar/users/aklie/data/datasets/HPAP/m

In [33]:
# Read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to make average prediction bigwigs over models\n# USAGE: sbatch \\\n#--job-name=average_predictions \\\n#--partition carter-compute\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=16G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#average_predictions.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate eugene_tools\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib\n\n# file lists\ncelltype={}\n{}\n{}\n{}\noutput_dir={}\n\n# echo the celltype and peak\necho -e "Celltype: $celltype"\necho -e "Bias preds: ${{bias_preds[@]}}"\necho -e "Chrombpnet nobias preds: ${{chrombpnet_nobias_preds[@]}}"\necho -e "Chrombpnet preds: ${{chrombpnet_preds[@]}}"\necho -e "Output directory: $output_dir\\n"\n\n# wigToBigWig\nwigToBigWig={}\nchromsizes={}\n\n# make the output directory\nmkdir -p $output_dir\n\n# Run cmd for bias. If there are multiple bias predictions, take the mean. Else, just copy the file\n

In [34]:
# Write the script for each celltype
for celltype, output_dirs_celltype in groups.items():
    bias_preds = [f"{outdir}/{celltype}_bias.bw" for outdir in output_dirs_celltype]
    chrombpnet_nobias_preds = [f"{outdir}/{celltype}_chrombpnet_nobias.bw" for outdir in output_dirs_celltype]
    chrombpnet_preds = [f"{outdir}/{celltype}_chrombpnet.bw" for outdir in output_dirs_celltype]
    output_dir = f"{path_out}/{celltype}/average/predictions"
    os.makedirs(output_dir, exist_ok=True)
    print(celltype)
    print(output_dir)
    bias_preds_str="bias_preds=(\n"
    for model in bias_preds:
        bias_preds_str += f"\t{model}\n"
    bias_preds_str += ")"
    print(bias_preds_str)
    chrombpnet_nobias_preds_str="chrombpnet_nobias_preds=(\n"
    for model in chrombpnet_nobias_preds:
        chrombpnet_nobias_preds_str += f"\t{model}\n"
    chrombpnet_nobias_preds_str += ")"
    print(chrombpnet_nobias_preds_str)
    chrombpnet_preds_str="chrombpnet_preds=(\n"
    for model in chrombpnet_preds:
        chrombpnet_preds_str += f"\t{model}\n"
    chrombpnet_preds_str += ")"
    print(chrombpnet_preds_str)
    with open(f"{path_scripts}/{name}_{celltype}_average_predictions.sh", "w") as f:
        f.write(template.format(
            celltype,
            bias_preds_str,
            chrombpnet_nobias_preds_str,
            chrombpnet_preds_str,
            output_dir,
            path_wigToBigWig,
            path_chromsizes
        ))

Schwann
/cellar/users/aklie/data/datasets/HPAP/models/Schwann/average/predictions
bias_preds=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/predictions/Schwann_bias.bw
)
chrombpnet_nobias_preds=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/predictions/Schwann_chrombpnet_nobias.bw
)
chrombpnet_preds=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/predictions/Schwann_chrombpnet.bw
)
Ductal
/cellar/users/aklie/data/datasets/HPAP/models/Ductal/average/predictions
bias_preds=(
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/predictions/Ductal_bias.bw
)
chrombpnet_nobias_preds=(
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/predictions/Ductal_chrombpnet_nobias.bw
)
chrombpnet_preds=(
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/predictions/Ductal_chrombpnet.bw
)
Acinar
/cellar/users/aklie/data/datasets/HPAP/mo

# Contributions

In [35]:
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/contributions.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/contributions"
os.makedirs(path_scripts, exist_ok=True)
name = "HPAP"

In [36]:
# Replace predictions with contributions
output_dirs = [f"{outdir.replace('predictions', 'contributions')}" for outdir in output_dirs]

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/contributions
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/contributions
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3/contributions
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3/contributions
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3/contributions
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3/contributions
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3/contributions
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3/contributions
	/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/chrombpnet/0.3/contributions
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/chrombpnet/0.3/contributions
	/cellar/users/aklie/data/datasets/HPAP/models/Immune/fold_0/chrombpnet/0.3/c

In [37]:
# read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to make contributions bigwigs with one or more chrombpnet models\n# USAGE: sbatch \\\n#--job-name=contributions \\\n#--account carter-gpu \\\n#--partition carter-gpu \\\n#--gpus=a30:1 \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=32G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#contributions.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import tensorflow as tf; print(tf.config.list_physical_devices(\'GPU\'))"\n\n# file lists\n{}\n{}\n{}\n{}\n\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\nchrombpnet_nobias_model=${{chrombpnet_nobias_models[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# echo the celltype and peak\necho -e "Celltype: $celltype"\necho -e "Peakset:

In [38]:
# Write the script
with open(f"{path_scripts}/{name}_contributions.sh", "w") as f:
    f.write(template.format(
        celltypes_str,
        peaks_str,
        chrombpnet_nobias_models_str,
        output_dirs_str,
        path_genome_fasta,
        path_chromsizes,
    ))

# Averaging contributions

In [39]:
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/average_contributions.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/contributions"
os.makedirs(path_scripts, exist_ok=True)
name = "HPAP"
window=400

In [40]:
# group outdirs by celltype
groups = {}
for outdir in output_dirs:
    group = outdir.split("/")[-5]
    if group not in groups:
        groups[group] = []
    groups[group].append(outdir)
groups

{'Schwann': ['/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/contributions'],
 'Ductal': ['/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/contributions'],
 'Acinar': ['/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3/contributions'],
 'Delta': ['/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3/contributions'],
 'Endothelial': ['/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3/contributions'],
 'Q_Stellate': ['/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3/contributions'],
 'A_Stellate': ['/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3/contributions'],
 'Gamma': ['/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3/contributions'],
 'Alpha': ['/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/chrombpnet/0.3/contributions'],
 'Beta': ['/cellar/users/aklie/da

In [41]:
# read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to compute averaged contributions and profiles across folds\n# USAGE: sbatch \\\n#--job-name=avg_contributions \\\n#--account=carter-compute \\\n#--output=slurm_logs/%x.%A.out \\\n#--mem=128G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#average_contributions.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\\\n"\n\n# Set up environment\nsource activate eugene_tools\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib\n\n# Define input files\ncelltype={}\n{}\n{}\n{}\n{}\noutput_dir={}\nwindow={}\n\n# Create output directory\nmkdir -p $output_dir\n\n# Define paths\nwigToBigWig={}\nchromsizes={}\npath_script={}\n\n# Compute mean bigwig for counts. If there are multiple bias predictions, take the mean. Else, just copy the file\nif [ ${{#counts[@]}} -eq 1 ]; then\n    cmd="cp ${{counts[0]}} $output_dir/${{celltype}}_counts.bw"\nelse\n    cmd="wiggletools mean ${{counts[@]}} > ${{celltype}}_temp_counts.wig && $wigToBigWig $

In [42]:
# Write the script for each celltype
for celltype, output_dirs_celltype in groups.items():
    counts = [f"{outdir}/{celltype}.counts_scores.bw" for outdir in output_dirs_celltype]
    profile = [f"{outdir}/{celltype}.profile_scores.bw" for outdir in output_dirs_celltype]
    counts_h5 = [f"{outdir}/{celltype}.counts_scores.h5" for outdir in output_dirs_celltype]
    profile_h5 = [f"{outdir}/{celltype}.profile_scores.h5" for outdir in output_dirs_celltype]
    output_dir = f"{path_out}/{celltype}/average/contributions"
    os.makedirs(output_dir, exist_ok=True)

    counts_str="counts=(\n"
    for model in counts:
        counts_str += f"\t{model}\n"
    counts_str += ")"
    print(counts_str)

    profile_str="profile=(\n"
    for model in profile:
        profile_str += f"\t{model}\n"
    profile_str += ")"
    print(profile_str)

    counts_h5_str="counts_h5=(\n"
    for model in counts_h5:
        counts_h5_str += f"\t{model}\n"
    counts_h5_str += ")"
    print(counts_h5_str)

    profile_h5_str="profile_h5=(\n"
    for model in profile_h5:
        profile_h5_str += f"\t{model}\n"
    profile_h5_str += ")"
    print(profile_h5_str)

    with open(f"{path_scripts}/{name}_{celltype}_average_contributions.sh", "w") as f:
        f.write(template.format(
            celltype,
            counts_str,
            profile_str,
            counts_h5_str,
            profile_h5_str,
            output_dir,
            window,
            path_wigToBigWig,
            path_chromsizes,
            path_contribution_averaging_script,
        ))

counts=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/contributions/Schwann.counts_scores.bw
)
profile=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/contributions/Schwann.profile_scores.bw
)
counts_h5=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/contributions/Schwann.counts_scores.h5
)
profile_h5=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/contributions/Schwann.profile_scores.h5
)
counts=(
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/contributions/Ductal.counts_scores.bw
)
profile=(
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/contributions/Ductal.profile_scores.bw
)
counts_h5=(
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/contributions/Ductal.counts_scores.h5
)
profile_h5=(
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/contributions

# De novo motif discovery

In [43]:
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/motif_discovery.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/motifs"
os.makedirs(path_scripts, exist_ok=True)
name = "HPAP"
n_seqlets = 100000
leiden_res = 2
window = 400

In [44]:
# read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to run de novo motif discovery on contributions\n# USAGE: sbatch \\\n#--job-name=motif_discovery \\\n#--partition=carter-compute \\\n#--output=slurm_logs/%x.%A.%a.out \\\n#--array=1-12%12 \\\n#--mem=128G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#motif_discovery.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set up environment\nsource activate eugene_tools\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/opt/miniconda3/lib/\nexport LD_LIBRARY_PATH=/cm/shared/apps/slurm/current/lib64\nNUMBA_NUM_THREADS=$SLURM_CPUS_PER_TASK\n\n# Define input files\n{}\n{}\n{}\n{}\nn_seqlets={}\nleiden_res={}\nwindow={}\n\n# Make the fold the slurm task id - 1\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\ninput_h5=${{input_h5s[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\ntype=${{types[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# Create output directory\nmkdir -p $output_dir\n\n# Run motifs cmd\ncmd="modisco motifs \\\n-i $input_h5 \\\n-n 

In [45]:
# Find each cell types averaged contributions
avg_contributions = []
for group in groups:
    avg_contributions.append(f"{path_out}/{group}/average/contributions/{group}_counts.h5")
    avg_contributions.append(f"{path_out}/{group}/average/contributions/{group}_profile.h5")

avg_contributions_str="input_h5s=(\n"
for model in avg_contributions:
    avg_contributions_str += f"\t{model}\n"
avg_contributions_str += ")"
print(avg_contributions_str)

input_h5s=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/average/contributions/Schwann_counts.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/average/contributions/Schwann_profile.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/average/contributions/Ductal_counts.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/average/contributions/Ductal_profile.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/average/contributions/Acinar_counts.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/average/contributions/Acinar_profile.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/average/contributions/Delta_counts.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/average/contributions/Delta_profile.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/average/contributions/Endothelial_counts.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/average/contributions/Endothelial_profile.h5
	/cellar/users/aklie/data/dataset

In [46]:
# celltypes x2
celltypes_str="celltypes=(\n"
for celltype in groups.keys():
    celltypes_str += f"\t{celltype}\n"
    celltypes_str += f"\t{celltype}\n"
celltypes_str += ")"
print(celltypes_str)

celltypes=(
	Schwann
	Schwann
	Ductal
	Ductal
	Acinar
	Acinar
	Delta
	Delta
	Endothelial
	Endothelial
	Q_Stellate
	Q_Stellate
	A_Stellate
	A_Stellate
	Gamma
	Gamma
	Alpha
	Alpha
	Beta
	Beta
	Immune
	Immune
	MUC5B_Ductal
	MUC5B_Ductal
)


In [47]:
# Make the output dirs x 2 
output_dirs = []
for group in groups:
    output_dirs.append(f"{path_out}/{group}/average/motifs")

output_dirs_str="output_dirs=(\n"
for model in output_dirs:
    output_dirs_str += f"\t{model}\n"
    output_dirs_str += f"\t{model}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/average/motifs
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/average/m

In [48]:
# get one of each of type: counts and profile
types = []
for group in groups:
    types.append("counts")
    types.append("profile")
types_str="types=(\n"
for model in types:
    types_str += f"\t{model}\n"
types_str += ")"
print(types_str)

types=(
	counts
	profile
	counts
	profile
	counts
	profile
	counts
	profile
	counts
	profile
	counts
	profile
	counts
	profile
	counts
	profile
	counts
	profile
	counts
	profile
	counts
	profile
	counts
	profile
)


In [49]:
# Write the script
with open(f"{path_scripts}/{name}_motif_discovery.sh", "w") as f:
    f.write(template.format(
        celltypes_str,
        avg_contributions_str,
        output_dirs_str,
        types_str,
        n_seqlets,
        leiden_res,
        window,
        path_meme_file,
        path_pfm_script,
    ))

# Motif clustering

In [50]:
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/motif_clustering.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/motifs"
os.makedirs(path_scripts, exist_ok=True)
name = "HPAP"
t = 0.999

In [51]:
# Find paths to pfms
paths_pfm = []
for group in groups:
    paths_pfm.append(f"{path_out}/{group}/average/motifs/pfms/counts/{group}.counts.pfm")
    paths_pfm.append(f"{path_out}/{group}/average/motifs/pfms/profile/{group}.profile.pfm")
paths_pfm_str="paths_pfm=(\n"
for model in paths_pfm:
    paths_pfm_str += f"\t{model}\n"
paths_pfm_str += ")"
print(paths_pfm_str)

paths_pfm=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/average/motifs/pfms/counts/Schwann.counts.pfm
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/average/motifs/pfms/profile/Schwann.profile.pfm
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/average/motifs/pfms/counts/Ductal.counts.pfm
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/average/motifs/pfms/profile/Ductal.profile.pfm
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/average/motifs/pfms/counts/Acinar.counts.pfm
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/average/motifs/pfms/profile/Acinar.profile.pfm
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/average/motifs/pfms/counts/Delta.counts.pfm
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/average/motifs/pfms/profile/Delta.profile.pfm
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/average/motifs/pfms/counts/Endothelial.counts.pfm
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/average/motifs/pfms/

In [52]:
# Read in template as a string
template = open(path_template).read()

In [53]:
# Write the script
with open(f"{path_scripts}/{name}_motif_clustering.sh", "w") as f:
    f.write(template.format(
        paths_pfm_str,
        path_out + "/motifs",
        t,
        path_meme_script,
        path_meme_file
    ))

# Create a master list of annotated motifs to use for hit calling

In [210]:
path_clustered_motifs = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/8_chrombpnet/rna_celltype_250k/motifs/cluster/cluster_key.txt"
path_initial_annotations = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/8_chrombpnet/rna_celltype_250k/motifs/tfs_initial.txt"

In [ ]:
initial_annotations = pd.read_csv(path_initial_annotations, sep="\t", header=None, names=["cluster_name", "tomtom_annotation"])
initial_annotations

In [ ]:
clustered_motifs = pd.read_csv(path_clustered_motifs, sep="\t", header=None, names=["cluster_name", "pfm_names"])
clustered_motifs["pfm_names"] = clustered_motifs["pfm_names"].str.split(",")
clustered_motifs = clustered_motifs.explode("pfm_names")
clustered_motifs["cell_type"] = clustered_motifs["pfm_names"].str.extract(r"(SC\..*?)\.")
clustered_motifs["model_type"] = clustered_motifs["pfm_names"].str.extract(r"\.(counts|profile)")
clustered_motifs["modisco_name"] = clustered_motifs["pfm_names"].str.split(".").str[-2]
clustered_motifs["modisco_name"] = "pos_patterns." + clustered_motifs["modisco_name"]
clustered_motifs = clustered_motifs.merge(initial_annotations, on="cluster_name", how="left")
clustered_motifs

In [ ]:
# Save tab-delimeted 2 column file for SC.beta counts [modisco_name, tomtom_annotation in 2nd]
subset = clustered_motifs[clustered_motifs["model_type"] == "counts"]
subset = subset[subset["cell_type"] == "SC.beta"]
subset["tomtom_annotation_number"] = subset.groupby("tomtom_annotation").cumcount().astype(str).replace("0", "")
msk = subset["tomtom_annotation_number"] != ""
subset.loc[msk, "tomtom_annotation"] = subset.loc[msk, "tomtom_annotation"] + "_" + subset.loc[msk, "tomtom_annotation_number"]
subset = subset[["modisco_name", "tomtom_annotation"]]
subset.to_csv("/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/results/8_chrombpnet/rna_celltype_250k/SC.beta/average/motifs/hits/counts/custom_motif_names.txt", sep="\t", header=False, index=False)

In [ ]:
subset["tomtom_annotation"].value_counts()

# Motif hit calling

In [54]:
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/motif_hits.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/motifs"
os.makedirs(path_scripts, exist_ok=True)
name = "HPAP"
t = 0.999

In [55]:
# Cell types
celltypes_str="celltypes=(\n"
for celltype in groups.keys():
    celltypes_str += f"\t{celltype}\n"
    celltypes_str += f"\t{celltype}\n"
celltypes_str += ")"
print(celltypes_str)

# Find peaks corresponding to modisco celltypes
peaks = []
for group in groups:
    peaks.append(narrowPeaks_dict[group])
    peaks.append(narrowPeaks_dict[group])
peaks_str="peaks=(\n"
for model in peaks:
    peaks_str += f"\t{model}\n"
peaks_str += ")"
print(peaks_str)

# Find each cell types averaged contributions
avg_contributions = []
for group in groups:
    avg_contributions.append(f"{path_out}/{group}/average/contributions/{group}_counts.h5")
    avg_contributions.append(f"{path_out}/{group}/average/contributions/{group}_profile.h5")
avg_contributions_str="input_h5s=(\n"
for model in avg_contributions:
    avg_contributions_str += f"\t{model}\n"
avg_contributions_str += ")"
print(avg_contributions_str)

# Find paths to modiscos
modiscos = []
for group in groups:
    modiscos.append(f"{path_out}/{group}/average/motifs/{group}.counts.modisco.h5")
    modiscos.append(f"{path_out}/{group}/average/motifs/{group}.profile.modisco.h5")
modiscos_str="modiscos=(\n"
for model in modiscos:
    modiscos_str += f"\t{model}\n"
modiscos_str += ")"
print(modiscos_str)

# types
types = []
for group in groups:
    types.append("counts")
    types.append("profile")
types_str="types=(\n"
for model in types:
    types_str += f"\t{model}\n"
types_str += ")"
print(types_str)

# output_dirs
output_dirs = []
for group in groups:
    output_dirs.append(f"{path_out}/{group}/average/motifs/hits/counts")
    output_dirs.append(f"{path_out}/{group}/average/motifs/hits/profile")
output_dirs_str="output_dirs=(\n"
for model in output_dirs:
    output_dirs_str += f"\t{model}\n"
output_dirs_str += ")"
print(output_dirs_str)

celltypes=(
	Schwann
	Schwann
	Ductal
	Ductal
	Acinar
	Acinar
	Delta
	Delta
	Endothelial
	Endothelial
	Q_Stellate
	Q_Stellate
	A_Stellate
	A_Stellate
	Gamma
	Gamma
	Alpha
	Alpha
	Beta
	Beta
	Immune
	Immune
	MUC5B_Ductal
	MUC5B_Ductal
)
peaks=(
	/cellar/users/aklie/data/datasets/HPAP/peaks/Schwann_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Schwann_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Ductal_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Ductal_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Acinar_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Acinar_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Delta_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Delta_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Endothelial_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/Endothelial_peaks.narrowPeak
	/cellar/users/aklie/data/datasets/HPAP/peaks/

In [56]:
# Read in template as a string
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to run motif hit calling\n# USAGE: sbatch \\\n#--job-name=motif_hits \\\n#--partition=carter-gpu \\\n#--account=carter-gpu \\\n#--gpus=a30:1 \\\n#--output=slurm_logs/%x.%A.%a.out \\\n#--mem=128G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#motif_hits.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Configure environment\nsource activate finemo_gpu\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/opt/miniconda3/envs/finemo_gpu/lib/\n\n# Define input files\n{}\n{}\n{}\n{}\n{}\n{}\nwindow={}\n\n# Make the fold the slurm task id - 1\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\npeak=${{peaks[$SLURM_ARRAY_TASK_ID - 1]}}\ninput_h5=${{input_h5s[$SLURM_ARRAY_TASK_ID - 1]}}\nmodisco=${{modiscos[$SLURM_ARRAY_TASK_ID - 1]}}\ntype=${{types[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# Create output directory\nmkdir -p $output_dir\n\n# Run cmd to preprocess\ncmd="finemo extract-regions-chrombpnet-h5 

In [57]:
# Write the script
with open(f"{path_scripts}/{name}_motif_hits.sh", "w") as f:
    f.write(template.format(
        celltypes_str,
        peaks_str,
        avg_contributions_str,
        modiscos_str,
        types_str,
        output_dirs_str,
        window,
    ))

# Variant scoring

In [58]:
# Paths
path_template = "/cellar/users/aklie/projects/ML4GLand/chrombpnet/templates/variant_scoring.txt"
path_scripts = "/cellar/users/aklie/data/datasets/HPAP/bin/chrombpnet/scripts/variant_scoring"
os.makedirs(path_scripts, exist_ok=True)

In [59]:
paths_variants = glob.glob("/cellar/users/aklie/data/ref/variants/*/*minimal.snps.chrombpnet*.bed")
paths_variants = [f for f in paths_variants if "TOPMed" not in f]
names_variants = ["T1D_Chiou_2021_cred_set", "eg_SuSIE_95_CS_meta_T1D_137_signals", "2hGlu_MAGIC", "HbA1c_MAGIC", "FG_MAGIC", "FI_MAGIC", "T2D_DIAMANTE_multiancestry", "ENCFF250UJY"]
dict(zip(names_variants, paths_variants))

{'T1D_Chiou_2021_cred_set': '/cellar/users/aklie/data/ref/variants/dbSNP/ENCFF250UJY.minimal.snps.chrombpnet.bed',
 'eg_SuSIE_95_CS_meta_T1D_137_signals': '/cellar/users/aklie/data/ref/variants/T1D/T1D_Chiou_2021_cred_set.wGWAS_info.minimal.snps.chrombpnet.bed',
 '2hGlu_MAGIC': '/cellar/users/aklie/data/ref/variants/T1D/eg_SuSIE_95_CS_meta_T1D_137_signals.minimal.snps.chrombpnet.bed',
 'HbA1c_MAGIC': '/cellar/users/aklie/data/ref/variants/MAGIC/2hGlu_MAGIC_trans_ancestry_pseudo_credset.LDproxyRsq0.8.wGWAS_info.minimal.snps.chrombpnet.bed',
 'FG_MAGIC': '/cellar/users/aklie/data/ref/variants/MAGIC/HbA1c_MAGIC_trans_ancestry_pseudo_credset.LDproxyRsq0.8.wGWAS_info.minimal.snps.chrombpnet.bed',
 'FI_MAGIC': '/cellar/users/aklie/data/ref/variants/MAGIC/FG_MAGIC_trans_ancestry_pseudo_credset.LDproxyRsq0.8.wGWAS_info.minimal.snps.chrombpnet.bed',
 'T2D_DIAMANTE_multiancestry': '/cellar/users/aklie/data/ref/variants/MAGIC/FI_MAGIC_trans_ancestry_pseudo_credset.LDproxyRsq0.8.wGWAS_info.minimal

In [60]:
# Update output_dirs to be 
output_dirs = []
for group in groups:
    for i in range(folds):
        output_dirs.append(f"{path_out}/{group}/fold_{i}/chrombpnet/{beta}")

# Same for output_dirs
output_dirs_str="output_dirs=(\n"
for output_dir in output_dirs:
    output_dirs_str += f"\t{output_dir}\n"
output_dirs_str += ")"
print(output_dirs_str)

output_dirs=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Beta/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/Immune/fold_0/chrombpnet/0.3
	/cellar/users/aklie/data/datasets/HPAP/models/MUC5B_Ductal/fold_0/chrombpnet/0.3
)


In [61]:
# Find chrombpnet_nobias models
chrombpnet_nobias_models = []
for outdir in output_dirs:
    f = glob.glob(f"{outdir}/models/chrombpnet_nobias.h5")[0]
    group = outdir.split("/")[-3]
    chrombpnet_nobias_models.append(f)

# Same for chrombpnet_nobias_models
chrombpnet_nobias_models_str="chrombpnet_nobias_models=(\n"
for model in chrombpnet_nobias_models:
    chrombpnet_nobias_models_str += f"\t{model}\n"
chrombpnet_nobias_models_str += ")"
print(chrombpnet_nobias_models_str)

chrombpnet_nobias_models=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/datasets/HPAP/models/Alpha/fold_0/chrombpnet/0.3/models/chrombpnet_nobias.h5
	/cellar/users/aklie/data/

In [62]:
# Define the groups to train models on
groups = list(fragments_dict.keys())

celltypes_str="celltypes=(\n"
for celltype in groups:
    for i in range(folds):
        celltypes_str += f"\t{celltype}\n"
celltypes_str += ")"
print(celltypes_str)

celltypes=(
	Schwann
	Ductal
	Acinar
	Delta
	Endothelial
	Q_Stellate
	A_Stellate
	Gamma
	Alpha
	Beta
	Immune
	MUC5B_Ductal
)


In [63]:
# Get template
template = open(path_template).read()
template

'#! /bin/bash\n\n#####\n# Script to get variant scores with a trained chrombpnet model\n# USAGE: sbatch \\\n#--job-name=variant_scoring \\\n#--account carter-gpu \\\n#--partition carter-gpu \\\n#--gpus=a30:1 \\\n#--output slurm_logs/%x.%A.%a.out \\\n#--mem=32G \\\n#-n 4 \\\n#-t 02-00:00:00 \\\n#--array=1-12%12 \\\n#variant_scoring.sh\n#####\n\ndate\necho -e "Job ID: $SLURM_JOB_ID\\n"\n\n# Set-up env\nsource activate chrombpnet\nexport LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/cellar/users/aklie/opt/miniconda3/envs/chrombpnet/lib\npython -c "import tensorflow as tf; print(tf.config.list_physical_devices(\'GPU\'))"\n\n# file lists\n{}\n{}\n{}\nlist={}\nrandom_seed={}\n\n# Grab each for this SLURM task\ncelltype=${{celltypes[$SLURM_ARRAY_TASK_ID - 1]}}\nchrombpnet_nobias_model=${{chrombpnet_nobias_models[$SLURM_ARRAY_TASK_ID - 1]}}\noutput_dir=${{output_dirs[$SLURM_ARRAY_TASK_ID - 1]}}\n\n# echo the celltype and peak\necho -e "Celltype: $celltype"\necho -e "Model path: $chrombpnet_nobias_model"\n

In [64]:
# for each variant list
for path_variant, name_variant in zip(paths_variants, names_variants):

    # Output dirs are the same as the chrombpnet_nobias models
    output_dirs_str="output_dirs=(\n"
    for output_dir in output_dirs:
        output_dirs_str += f"\t{output_dir}/variant_scoring/{name_variant}\n"
    output_dirs_str += ")"
    print(output_dirs_str)

    # Write the script
    with open(f"{path_scripts}/{name}_{name_variant}_variant_scoring.sh", "w") as f:
        f.write(template.format(
            celltypes_str,
            chrombpnet_nobias_models_str,
            output_dirs_str,
            path_variant,
            1234,
            path_variant_scorer,
            path_genome_fasta,
            path_chromsizes,
        ))

output_dirs=(
	/cellar/users/aklie/data/datasets/HPAP/models/Schwann/fold_0/chrombpnet/0.3/variant_scoring/T1D_Chiou_2021_cred_set
	/cellar/users/aklie/data/datasets/HPAP/models/Ductal/fold_0/chrombpnet/0.3/variant_scoring/T1D_Chiou_2021_cred_set
	/cellar/users/aklie/data/datasets/HPAP/models/Acinar/fold_0/chrombpnet/0.3/variant_scoring/T1D_Chiou_2021_cred_set
	/cellar/users/aklie/data/datasets/HPAP/models/Delta/fold_0/chrombpnet/0.3/variant_scoring/T1D_Chiou_2021_cred_set
	/cellar/users/aklie/data/datasets/HPAP/models/Endothelial/fold_0/chrombpnet/0.3/variant_scoring/T1D_Chiou_2021_cred_set
	/cellar/users/aklie/data/datasets/HPAP/models/Q_Stellate/fold_0/chrombpnet/0.3/variant_scoring/T1D_Chiou_2021_cred_set
	/cellar/users/aklie/data/datasets/HPAP/models/A_Stellate/fold_0/chrombpnet/0.3/variant_scoring/T1D_Chiou_2021_cred_set
	/cellar/users/aklie/data/datasets/HPAP/models/Gamma/fold_0/chrombpnet/0.3/variant_scoring/T1D_Chiou_2021_cred_set
	/cellar/users/aklie/data/datasets/HPAP/models

# DONE!

---

In [59]:
folds = 1
celltypes = []
peaks = []
output_dirs = []
fold_lst = []
for group in groups:
    for i in range(folds):
        # Make a fold_{fold} directory
        outdir = f"{path_out}/{group}/fold_{i}/negatives"
        os.makedirs(outdir, exist_ok=True)
        peakset = narrowPeaks[group]
        celltypes.append(group)
        peaks.append(peakset)
        output_dirs.append(outdir)
        fold_lst.append(i)

In [60]:
# Print out celltypes in same way I would write a bash array so I can copy paste into a script
print("celltypes=(")
for celltype in celltypes:
    print(f"    {celltype}")
print(")")

In [61]:
# Samme for peaks
print("peaks=(")
for peak in peaks:
    print(f"    {peak}")
print(")")

In [62]:
# Samme for output_dirs
print("output_dirs=(")
for outdir in output_dirs:
    print(f"    {outdir}")
print(")")

In [63]:
#  Samme for fold_lst
print("folds=(")
for fold in fold_lst:
    print(f"    {fold}")
print(")")

# Bias models

In [72]:
# Find negatives
negatives = {}
for outdir in output_dirs:
    group = outdir.split("/")[-3]
    negatives[group] = f"{outdir}/{group}_negatives.bed"

In [76]:
# Update output_dirs to replace "negatives" with bias_model/$beta
output_dirs = [f"{outdir.replace("negatives/", "")}/bias_model/$beta" for outdir in output_dirs]

In [77]:
# Same for neegatives
print("negatives=(")
for negative in negatives.values():
    print(f"    {negative}")
print(")")

In [78]:
# Same for output_dirs
print("output_dirs=(")
for outdir in output_dirs:
    print(f"    {outdir}")
print(")")

In [79]:
# Same for fragments
print("fragments=(")
for fragment in fragments.values():
    print(f"    {fragment}")
print(")")

# Prep dataset config

In [7]:
# path to template config file
path_config = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/bin/8_sequence_models/configs/prep_dataset_template.yaml"

In [8]:
# load template config
config = yaml.safe_load(open(path_config, 'r'))
config

In [9]:
# Only use these signal betas
signal_betas = [0.5]

In [25]:
config_paths = []
for i, group in enumerate(groups):
    for signal_beta in signal_betas:
        out_dir = os.path.join(path_out, group, "prep_dataset", f"{signal_beta}")
        os.makedirs(out_dir, exist_ok=True)
        curr_out = os.path.join(out_dir, f"{group}.yaml")
        curr_config = config.copy()
        curr_config["name"] = group
        curr_config["seqdata"]["bws"] = [unstranded_bws[group]]
        curr_config["seqdata"]["bw_names"] = [group]
        curr_config["seqdata"]["loci"] = narrowPeaks[celltypes[i]]
        curr_config["negatives"]["signal"] = curr_config["seqdata"]["bws"][0]
        curr_config['negatives']['signal_beta'] = round(float(signal_beta), 1)
        with open(curr_out, 'w') as f:
            yaml.dump(curr_config, f)
        print(curr_out)
        config_paths.append(curr_out)

In [27]:
# Make a dataframe with paths of configs in first column and output directory in second column
df = pd.DataFrame()
df["config"] = config_paths
df["out_dir"] = [os.path.dirname(p) for p in config_paths]
df.to_csv("/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/bin/8_sequence_models/metadata/prep_dataset_celltype+condition+timepoint.tsv", sep="\t", index=False, header=False)

# Bpnet fit configs

In [12]:
# path to template config file
path_config = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/bin/8_sequence_models/configs/bpnet_fit_template.yaml"

In [13]:
# load template config
config = yaml.safe_load(open(path_config, 'r'))
config

In [16]:
# Grab paths of seqdatas from the previous step
seqdatas = {}
for f in glob.glob(os.path.join(path_out, "*", "prep_dataset", "*", "*.minimal.seqdata")):
    signal_beta = float(f.split("/")[-2])
    group = os.path.basename(f).split(".minimal.seqdata")[0]
    if group not in seqdatas:
        seqdatas[group] = {}
    seqdatas[group][signal_beta] = f
seqdatas

In [17]:
# How many folds?
folds = 5

In [18]:
# Which signal betas to use?
signal_betas = [0.5]

In [21]:
# Make config files
config_paths = []
for i, group in enumerate(groups):
    if group not in seqdatas:
        continue
    for signal_beta in signal_betas:
        for fold in range(folds):
            out_dir = os.path.join(path_out, group, "bpnet_fit", f"fold_{fold}", f"{signal_beta}")
            os.makedirs(out_dir, exist_ok=True)
            curr_out = os.path.join(out_dir, f"{group}_fold_{fold}.yaml")
            curr_config = config.copy()
            curr_config["name"] = group
            curr_config["seqdata"]["path"] = seqdatas[group][signal_beta]
            curr_config["seqdata"]["fold"] = f"fold_{fold}"
            with open(curr_out, 'w') as f:
                yaml.dump(curr_config, f)
            print(curr_out)
            config_paths.append(curr_out)

In [22]:
# Keep only fold_0 configs
config_paths = sorted([c for c in config_paths if "fold_0" in c])
config_paths

In [23]:
# Keep only SC.alpha cellypes
#config_paths = [c for c in config_paths if "SC.alpha" in c]
config_paths

In [25]:
# Make a dataframe with paths of configs in first column and output directory in second column
df = pd.DataFrame()
df["config"] = config_paths
df["out_dir"] = [os.path.dirname(p) for p in config_paths]
df.to_csv("/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/bin/8_sequence_models/metadata/bpnet_fit_celltype+condition+timepoint.tsv", sep="\t", index=False, header=False)

# Bias fit configs

In [16]:
# path to template config file
path_config = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/bin/9_sequence_models/configs/bias_fit_template.yaml"

In [17]:
# load template config
config = yaml.safe_load(open(path_config, 'r'))
config

In [18]:
seqdatas = {}
for f in glob.glob(os.path.join(path_out, "*", "prep_dataset", "*", "*.minimal.seqdata")):
    signal_beta = float(f.split("/")[-2])
    celltype = os.path.basename(f).split(".minimal.seqdata")[0]
    if celltype not in seqdatas:
        seqdatas[celltype] = {}
    seqdatas[celltype][signal_beta] = f
seqdatas

In [19]:
# How many folds?
folds = 5

In [20]:
# Need to pull out Max Negative Counts: value from the following
reports = {}
for f in glob.glob(os.path.join(path_out, "*", "prep_dataset", "*", "*.report.html"), recursive=True):
    signal_beta = float(f.split("/")[-2])
    celltype = os.path.basename(f).split(".report.html")[0]
    with open(f, 'r') as f:
        content = f.read()
        max_neg_counts = float(content.split("Max Negative Counts: ")[1].split(",")[0])
    if celltype not in reports:
        reports[celltype] = {}
    reports[celltype][signal_beta] = max_neg_counts        
reports

In [21]:
signal_betas = [0.5]

In [22]:
config_paths = []
for i, celltype in enumerate(celltypes):
    for signal_beta in signal_betas:
        for fold in range(folds):
            out_dir = os.path.join(path_out, celltype, "bias_fit", f"fold_{fold}", f"{signal_beta}")
            os.makedirs(out_dir, exist_ok=True)
            curr_out = os.path.join(out_dir, f"{celltype}_fold_{fold}.yaml")
            curr_config = config.copy()
            curr_config["name"] = celltype
            curr_config["seqdata"]["path"] = seqdatas[celltype][signal_beta]
            curr_config["seqdata"]["fold"] = f"fold_{fold}"
            config["seqdata"]["max_counts"] = reports[celltype][signal_beta]
            with open(curr_out, 'w') as f:
                yaml.dump(curr_config, f)
            print(curr_out)
            config_paths.append(curr_out)

In [58]:
# Keep only fold_0 configs
#config_paths = sorted([c for c in config_paths if "fold_0" in c])
config_paths

In [61]:
# Make a dataframe with paths of configs in first column and output directory in second column
df = pd.DataFrame()
df["config"] = config_paths
df["out_dir"] = [os.path.dirname(p) for p in config_paths]
df.to_csv("/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/bin/9_sequence_models/metadata/bias_fit_0.5.tsv", sep="\t", index=False, header=False)

# Chrombpnet fit configs

In [23]:
# path to template config file
path_config = "/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/bin/9_sequence_models/configs/chrombpnet_fit_template.yaml"

In [24]:
# load template config
config = yaml.safe_load(open(path_config, 'r'))
config

In [25]:
# Grab paths of seqdatas from the previous step
seqdatas = {}
for f in glob.glob(os.path.join(path_out, "*", "prep_dataset", "*", "*.minimal.seqdata")):
    signal_beta = float(f.split("/")[-2])
    celltype = os.path.basename(f).split(".minimal.seqdata")[0]
    if celltype not in seqdatas:
        seqdatas[celltype] = {}
    seqdatas[celltype][signal_beta] = f
seqdatas

In [26]:
# Need to pull
bias_models = {}
paths = glob.glob(os.path.join(path_out, "*", "bias_fit", "fold_*", "*", "*.torch"), recursive=True)
paths = [p for p in paths if "final" not in p]
for f in paths:
    
    # Grab celltype
    celltype = os.path.basename(f).split(".torch")[0]
    if celltype not in bias_models:
        bias_models[celltype] = {}
    
    # Grab fold
    fold = f.split("/")[-3]
    if fold not in bias_models[celltype]:
        bias_models[celltype][fold] = {}

    # Grab signal_beta
    signal_beta = float(f.split("/")[-2])
    bias_models[celltype][fold][signal_beta] = f
    
bias_models

In [27]:
# How many folds?
folds = 5

In [28]:
# Which signal betas to use?
signal_betas = [0.5]

In [33]:
config_paths = []
for i, celltype in enumerate(celltypes):
    for fold in bias_models[celltype]:
        for signal_beta in signal_betas:
            out_dir = os.path.join(path_out, celltype, "chrombpnet_fit", fold, f"{signal_beta}")
            os.makedirs(out_dir, exist_ok=True)
            curr_out = os.path.join(out_dir, f"{celltype}_{fold}.yaml")
            curr_config = config.copy()
            curr_config["name"] = celltype
            curr_config["seqdata"]["path"] = seqdatas[celltype][signal_beta]
            curr_config["seqdata"]["fold"] = fold
            curr_config["model"]["bias_model"] = bias_models[celltype][fold][signal_beta]
            with open(curr_out, 'w') as f:
                yaml.dump(curr_config, f)
            print(curr_out)
            config_paths.append(curr_out)

In [34]:
# Keep only fold_0 configs
#config_paths = sorted([c for c in config_paths if "fold_0" in c])
config_paths

In [35]:
# Make a dataframe with paths of configs in first column and output directory in second column
df = pd.DataFrame()
df["config"] = config_paths
df["out_dir"] = [os.path.dirname(p) for p in config_paths]
df.to_csv("/cellar/users/aklie/data/datasets/igvf_sc-islet_10X-Multiome/bin/9_sequence_models/metadata/chrombpnet_fit_0.5.tsv", sep="\t", index=False, header=False)

# DONE!

---